In [1]:
api_v2 = bitfinex.bitfinex_v2.api_v2()

NameError: name 'bitfinex' is not defined

In [2]:
def getPercentChange_tf15m(df):
    #df['Date'] = df['Date'].apply(lambda x: dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['1h_change'] = 0
    df['4h_change'] = 0
    df['24h_change'] = 0
    if int(str(df_large['Date'][1]-df_large['Date'][0])[:1]) == 1:
        period = 1*24*60
    else:
        period_hr = int(str(df_large['Date'][1]-df_large['Date'][0])[7:9])
        period_m = int(str(df_large['Date'][1]-df_large['Date'][0])[10:12])
        if period_hr == 0:
            period = period_m
        else:
            period = period_hr * 60
    print(f'Time period = {period} minutes')
    
    if period <= 60:
        df['1h_change'] = ((df.Close - df.Close.shift(int(60/period)))/df.Close)
    if period <= 4*60:
        df['4h_change'] = ((df.Close - df.Close.shift(int(240/period)))/df.Close)
    if period <= 1440:
        df['24h_change'] = ((df.Close - df.Close.shift(int(1440/period)))/df.Close)
    df = df[(int(1440/period)):len(df)-int(1440/period)]
    df = df.reset_index(drop=True)
    return df

In [ ]:
def fixDataset(df,period:int,list_fixers,fill_missing=False,verbose=True):
    if type(df.Date.values[0]) == str:
        if verbose:
            print('Convert string datetime to datetime type')
        df['Date'] = df['Date'].apply(lambda x: dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    full_length = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in 
       datetime_range(df.iloc[0]['Date'], df.iloc[-1]['Date'], 
       timedelta(minutes=period))]
    full_length = pd.Series(full_length)
    full_length = full_length.apply(lambda x:dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    if verbose:
        print('Dataset size = {}'.format(len(df)))
        print('Proper size = {}'.format(len(full_length)))
    mask = full_length.isin(df['Date'].values)
    missing_date = full_length[~mask]
    if verbose:
        print('Number of missing data : {}'.format(len(missing_date)))
    
#     if len(full_length) != 0:
#         print('This dataset has missing date')
#         return missing_date,full_length
#     else:
#         return None
    full_length = pd.DataFrame(full_length)
    full_length.columns=['Date']
    merged = full_length.merge(df, how='left', on='Date')
    processed_fixer = []
    for fixer in list_fixers:
        processing = full_length.merge(fixer, how='left', on = 'Date')
        processed_fixer.append(processing)
    for i,fixer in enumerate(processed_fixer):
        merged = merged.fillna(fixer)
        if verbose:
            print('Fixed with fixer {} : Missing data is {}'.format(i+1,len(merged[merged.Open.isna()])))
    if fill_missing:
        for column in merged.keys():
            merged[column] = merged[column].interpolate(method ='linear', limit_direction ='forward')
        if verbose:
            print('Filled missing : Missing data is {}'.format(len(merged[merged.Open.isna()])))
    else:
        if verbose:
            print('Not filled : Missing data is {}'.format(len(merged[merged.Open.isna()])))
    print('Done')
    
    return merged

In [ ]:
def getAccurateLargeDataset(pair,period,year_start,year_stop,limit):
    is_not_accurate = True
    round = 0
    while is_not_accurate:
        dataset = getLargeDataset(pair,period,year_start,year_stop,limit)
        print('Round {} : Already get dataset.'.format(round+1))
        dts = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(dataset.iloc[0]['Date'], \
                                    dataset.iloc[-1]['Date'], timedelta(minutes=\
                                                                       int(period[:-1])))]
        dts = pd.Series(dts)
        print(len(dts))
        dts = dts.apply(lambda x:dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        mask = dts.isin(dataset['Date'].values)
        not_found = dts[~mask]
        print('Length : \nDataset = {}\nDate missed = {} Target = {}'.format(len(dataset),len(not_found),len(dts)+1))
        
        if len(dataset) == len(dts)+1:
            print('Done')
            return dataset
        else:
            print('Failed')

In [1]:
#get significant change candle mark
def getSignificantChange(df,threshold,verbose=False):
    df['%Change'] = abs(((df.Open-df.Close)/df.Close)*100)
    df['Percentile'] = df['%Change'].rank(pct=True)
    df['Significant_Change'] = df['Percentile'].apply(lambda x: 1 if x>threshold else 0)
    df_iter = df[df.Significant_Change == 1]
    if verbose:
        for i in range(len(df_iter)):
            print('At {} Change {}'.format(df_iter.iloc[i]['Date'],\
                                           df_iter.iloc[i]['Close']-df_iter.iloc[i]['Open']))
    df = df.drop(labels = ['%Change','Percentile'],axis=1)

    return df

In [ ]:
def detectSupportAndResistanceLines(df,wide_const,gap,n1,n2):
    #find reversal point
    support_resistance = []
    #support
    for candle in range(n1,len(df)-n2):
        drop_pass = True
        rise_pass = True
        is_support = False
        is_resistance = False
        
        for i in range(candle-n1+1,candle+1):
            if df.Low[i] > df.Low[i-1]:
                drop_pass = False
        for i in range(candle+1,candle+n2+1):
            if df.Low[i] < df.Low[i-1]:
                rise_pass = False
        if drop_pass and rise_pass:
            is_support = True
        
        drop_pass = True
        rise_pass = True
        
        for i in range(candle-n1+1,candle+1):
            if df.High[i] < df.High[i-1]:
                rise_pass = False
        for i in range(candle+1,candle+n2-1):
            if df.High[i] > df.High[i-1]:
                drop_pass = False
        if drop_pass and rise_pass:
            is_resistance = True
        
        if is_support or is_resistance:
            support_resistance.append((candle,df.Close[candle]))
            
                
    #find frequency
    gap = gap
    dict_freq = {}
    for i in range(1000,70000,gap):
        print(i)
        mid_line = i
        upper_bound = i + (gap*wide_const)
        lower_bound = i - (gap*wide_const)
        list_checking_lines = [mid_line,upper_bound,lower_bound]
        for line in list_checking_lines:
            df['hit_line'] = 0
            df.loc[(df['High']>= line) & (df['Low']<= line),['hit_line']] = 1
        dict_freq[mid_line] = len(df[df.hit_line!=0])
    return df,dict_freq,support_resistance
        

In [ ]:
def eliminateLoseSpot(df,result_column,signal_column,failed_buy_column):
#     #2
#     Lose Trade
    for i in range(len(df)):
        if df.iloc[i][result_column] == 'Lose':
            df.loc[df.iloc[i][failed_buy_column],[signal_column]] = 'Hold'
            df.loc[i,[signal_column]] = 'Hold'
    return df

In [ ]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [ ]:
#pair = Currency pair
#period = Timeframe
#start = List of start date in format [year,month,day,hour,minute]
#stop = List of stop date in format [year,month,day,hour,minute]
#limit = Limit size of dataset

In [ ]:
def getDataset(pair,period,start,stop,limit):
    
    t_start = datetime.datetime(start[0],start[1],start[2],start[3],start[4])
    t_start = time.mktime(t_start.timetuple()) * 1000

    t_stop = datetime.datetime(stop[0],stop[1],stop[2],stop[3],stop[4])
    t_stop = time.mktime(t_stop.timetuple()) * 1000
    
    result = api_v2.candles(symbol=pair,interval=period,limit=limit,start=t_start,end=t_stop)
    names = ['Date','Open','Close','High','Low','Volume']
    df = pd.DataFrame(result, columns=names)
    df['Date'] = pd.to_datetime(df['Date'],unit='ms')
    df = df.sort_values('Date',ascending=True)
    df = df.reset_index(drop=True)
    return df

In [ ]:
def getLargeDataset(pair,period,year_start,year_stop,limit):
    round = 1
    for i in range(year_start,year_stop+1):
        if i == 2021:
            month_stop = 7
        else:
            month_stop = 12
        for j in range(1,month_stop+1):
            if round == 1: 
                df = getDataset(pair,period,(i,j,1,0,0),(i,j+1,1,0,0),limit)
                round += 1
            elif j == 12:
                df_temp = getDataset(pair,period,(i,j,1,0,0),(i,j,31,0,0),limit)
                df = pd.concat([df[:-1],df_temp],ignore_index = True)
                round += 1
            else:
                df_temp = getDataset(pair,period,(i,j,1,0,0),(i,j+1,1,0,0),limit)
                df = pd.concat([df[:-1],df_temp],ignore_index = True)
                round += 1
    return df

In [ ]:
def getLargeDataset_1m(pair,period,year_start,year_stop,limit):
    round = 1
    month_days = [[2021,7,31]]
    for date in month_days:
        for j in range(1,date[2]+1):
            if round == 1:
                df = getDataset(pair,period,(date[0],date[1],j,0,0),(date[0],date[1],j,23,59),limit)
                print(df)
                round += 1
            else:
                df_temp = getDataset(pair,period,(date[0],date[1],j,0,0),(date[0],date[1],j,23,59),limit)
                #print(df_temp)
                df = pd.concat([df,df_temp],ignore_index = True)
                round += 1
            
    return df
            

In [4]:
def generateBuySellPoint(df,detect_size,thresh_const,spread_size=0,rsi_filter=False,\
                         rsi_buy=0.4,rsi_sell=0.5):
    size = detect_size
    threshold = int(thresh_const*detect_size)
    df['Buy/Sell_temp'] = np.empty((len(df), 0)).tolist()
    for i in range(len(df)-size+1):
        temp_df = df.iloc[i:i+size,:]
        df.loc[temp_df['Close'].idxmax(),['Buy/Sell_temp']].apply(lambda x: x.append('Sell'))
        df.loc[temp_df['Close'].idxmin(),['Buy/Sell_temp']].apply(lambda x: x.append('Buy'))
        #print(temp_df)
    #df['Buy/Sell'] = np.empty((len(df), 0)).tolist()
    df['Signal_temp'] = df['Buy/Sell_temp'].apply(lambda x: list(set(x))[0] if len(x)!=0 else 'Hold')
    df['Signal'] = 'Hold'
    df['Frequency'] = df['Buy/Sell_temp'].apply(lambda x: len(x) if len(x)>threshold else 0)
    df.loc[df.Frequency <= threshold,['Signal_temp']] = 'Hold'
    df['Signal_add'] = 'Hold'
    if spread_size != 0:
        for i in df.index:
            if df.iloc[i]['Signal_temp'] == 'Buy' or df.iloc[i]['Signal_temp'] == 'Sell':
                if i > spread_size-1:
                    df.loc[i-spread_size+1:i,['Signal_add']] = df.iloc[i]['Signal_temp']
                    df.loc[i+1:i+spread_size+1,['Signal_add']] = df.iloc[i]['Signal_temp']
                elif i < (len(df)-spread_size-1):
                    df.loc[i-spread_size+1:i,['Signal_add']] = df.iloc[i]['Signal_temp']
                    df.loc[i+1:i+spread_size+1,['Signal_add']] = df.iloc[i]['Signal_temp']
                elif i < spread_size-1:
                    df.loc[i+1:i+spread_size+1,['Signal_add']] = df.iloc[i]['Signal_temp']
                elif i > (len(df)-spread_size-1):
                    df.loc[i-spread_size+1:i,['Signal_add']] = df.iloc[i]['Signal_temp']
        df.loc[df['Signal_add'] != 'Hold', ['Signal']] =\
            df.loc[df['Signal_add'] != 'Hold', ['Signal_add']].values
    else:
        df['Signal'] = df['Signal_temp']
        print("Done")
    if rsi_filter:
        print('Before filter\nBuy Points : {}\nSell Points : {}'.format(\
                                                                len(df[df.Signal == 'Buy']),len(df[df.Signal == 'Sell'])))
        for i in range(len(df)):
            if df.Signal[i] == 'Buy':
                if df.RSI[i] >= rsi_buy*100:
                    df.loc[i,['Signal']] = 'Hold'
            elif df.Signal[i] == 'Sell':
                if df.RSI[i] <= rsi_sell*100:
                    df.loc[i,['Signal']] = 'Hold'
        print('After filter\nBuy Points : {}\nSell Points : {}'.format(\
                                                                len(df[df.Signal == 'Buy']),len(df[df.Signal == 'Sell'])))
            
    return df
    
    

In [ ]:
def recognize_candlestick(df):
    """
    Recognizes candlestick patterns and appends 2 additional columns to df;
    1st - Best Performance candlestick pattern matched by www.thepatternsite.com
    2nd - # of matched patterns
    """

    op = df['Open'].astype(float)
    hi = df['High'].astype(float)
    lo = df['Low'].astype(float)
    cl = df['Close'].astype(float)

    candle_names = talib.get_function_groups()['Pattern Recognition']

    # patterns not found in the patternsite.com
    exclude_items = ('CDLCOUNTERATTACK',
                     'CDLLONGLINE',
                     'CDLSHORTLINE',
                     'CDLSTALLEDPATTERN',
                     'CDLKICKINGBYLENGTH')

    candle_names = [candle for candle in candle_names if candle not in exclude_items]


    # create columns for each candle
    for candle in candle_names:
        # below is same as;
        # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
        df[candle] = getattr(talib, candle)(op, hi, lo, cl)


    df['candlestick_pattern'] = np.nan
    df['candlestick_match_count'] = np.nan
    for index, row in df.iterrows():

        # no pattern found
        if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
            df.loc[index,'candlestick_pattern'] = "NO_PATTERN"
            df.loc[index, 'candlestick_match_count'] = 0
        # single pattern found
        elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
            # bull pattern 100 or 200
            if any(row[candle_names].values > 0):
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
            # bear pattern -100 or -200
            else:
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
        # multiple patterns matched -- select best performance
        else:
            # filter out pattern names from bool list of values
            patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
            container = []
            for pattern in patterns:
                if row[pattern] > 0:
                    container.append(pattern + '_Bull')
                else:
                    container.append(pattern + '_Bear')
            rank_list = [candle_rankings[p] for p in container]
            if len(rank_list) == len(container):
                rank_index_best = rank_list.index(min(rank_list))
                df.loc[index, 'candlestick_pattern'] = container[rank_index_best]
                df.loc[index, 'candlestick_match_count'] = len(container)
    # clean up candle columns
    cols_to_drop = candle_names + list(exclude_items)
    df = df.drop(labels=candle_names,axis=1)
    df = pd.concat([df,pd.get_dummies(df.candlestick_pattern)],axis=1)
    #df.drop(cols_to_drop, axis = 1, inplace = True)
    return df

In [ ]:
def visualizeSignal(df,start,x_col,y_col,signal_col,buy_str,sell_str):
    df_temp = df[int(start*len(df)):]
    plt.plot(df_temp[x_col],df_temp[y_col])
    plt.scatter(df_temp.loc[df_temp[signal_col] == buy_str, x_col]\
                .values,df_temp.loc[df_temp[signal_col] == buy_str,y_col].values\
                , label='Buy Spot', color='green', s=25, marker="^")
    plt.scatter(df_temp.loc[df_temp[signal_col] == sell_str, x_col]\
                .values,df_temp.loc[df_temp[signal_col] == sell_str, y_col].values\
                , label='Sell Spot', color='red', s=25, marker="^")

In [ ]:
def visualizeProportion(dataframe,num_group,x_col,y_col,signal_col,buy_str,sell_str,max_show,\
                        random_show=False):
    if random_show:
        randomed_index_show = random.sample(range(0, num_group), max_show)
        list_df_splitted = np.array_split(dataframe,num_group)
        list_df_splitted = [list_df_splitted[i] for i in randomed_index_show]
    else:
        list_df_splitted = np.array_split(dataframe,num_group)
    for i,df_temp in enumerate(list_df_splitted[:max_show]):
        plt.plot(df_temp[x_col],df_temp[y_col])
        plt.scatter(df_temp.loc[df_temp[signal_col] == buy_str, x_col]\
                .values,df_temp.loc[df_temp[signal_col] == buy_str,y_col].values\
                , label='Buy Spot', color='green', s=40, marker="^")
        plt.scatter(df_temp.loc[df_temp[signal_col] == sell_str, x_col]\
                .values,df_temp.loc[df_temp[signal_col] == sell_str, y_col].values\
                , label='Sell Spot', color='red', s=40, marker="^")
        if buy_str not in [1,2,3] or sell_str not in [1,2,3]:
            plt.scatter(df_temp.loc[df_temp[signal_col] == 'Win', x_col]\
                    .values,df_temp.loc[df_temp[signal_col] == 'Win', y_col].values\
                    , label='Win Spot', color='magenta', s=40, marker="^")
            plt.scatter(df_temp.loc[df_temp[signal_col] == 'Lose', x_col]\
                    .values,df_temp.loc[df_temp[signal_col] == 'Lose', y_col].values\
                    , label='Lose Spot', color='black', s=40, marker="^")
        if random_show:
            plt.xlabel('Index' + str(randomed_index_show[i]), fontsize=18)
        else:
            plt.xlabel('Index' + str(i), fontsize=18)
        plt.legend()
        plt.show()
        

In [ ]:
def getRSI(df,lookback,price_col):
    rsi = talib.RSI(df[price_col],timeperiod=lookback)
    df = pd.concat([df, rsi], axis=1)
    df = df[lookback:].reset_index(drop=True)
    df = df.rename(columns={0:'RSI'})
    return df

In [ ]:
def getEMA50_200(df,price_col):
    df['EMA200'] = talib.EMA(df[price_col],timeperiod=200)
    df['EMA50'] = talib.EMA(df[price_col],timeperiod=50)
    df = df[200:].reset_index(drop=True)
    df['PercentDifPrice'] = ((df['EMA50'] - df['EMA200'])/df['Close'])*100
    df['EMA_Cross'] = np.where(df['EMA50'] >= df['EMA200'], 1, 0)
    df['PriceBelow200'] = np.where(df['Close'] <= df['EMA200'],0,1)
    df['PriceBelow50'] = np.where(df['Close'] <= df['EMA50'],0,1)
    df['EMA50Below200'] = np.where(df['EMA50'] <= df['EMA200'],0,1)
    return df

In [ ]:
def getCleanedDataset(df):
    df_with_date = df.reindex(columns=['Date','Open','Close','High','Low','Volume','RSI','Signal'\
                                       ,'PercentDifPrice','EMA_Cross','PriceBelow200','PriceBelow50'])
    df = df.loc[:,columns_selected3]
    df = df.reindex(columns=columns_reindexed3)
    return df,df_with_date

In [ ]:
def scaledDataset(df):
    
    df[['Open','Close','High','Low']]=\
                scaler_price.fit_transform(np.array(df[['Open','Close','High','Low']]).reshape(-1,4))
    df['RSI'] = scaler_rsi.fit_transform(np.array(df['RSI']).reshape(-1,1))
    #df['PercentDifPrice'] = scaler_percentdif.fit_transform(np.array(df['PercentDifPrice']).reshape(-1,1))
    #df['1h_change'] = scaler_1h.fit_transform(np.array(df['1h_change']).reshape(-1,1))
    #df['4h_change'] = scaler_4h.fit_transform(np.array(df['4h_change']).reshape(-1,1))
    #df['24h_change'] = scaler_24h.fit_transform(np.array(df['24h_change']).reshape(-1,1))
    df['Volume']=scaler_volume.fit_transform(np.array(df['Volume']).reshape(-1,1))
    df['Signal'] = df['Signal'].map({'Hold':0, 'Buy':1,'Sell':2})
    #df = np.array(df)
    return df

In [ ]:
def neccessaryFeatures(df):
    df = df.loc[:,columns_final4]
    return df

In [ ]:
def df_to_array(df):
    df = np.array(df)
    return df

In [ ]:
def createDataset(dataset, timestep = 1):
    dataX,dataY=[],[]
    for i in range(len(dataset)-timestep-1):
        a = dataset[i:(i+timestep),:-1]
        #a = a.reshape(-1)
        dataX.append(a)
        dataY.append(dataset[i+timestep,-1])
    return np.array(dataX).astype(float), np.array(dataY).astype(float)

In [ ]:
def testProfit(df,budget,confident,enable_stop_loss=False,calculate_fee=True):
    #buy, hold and sell only
    if calculate_fee:
        fee = 0.001
    else:
        fee = 0
    #df['Date'] = pd.to_datetime(df.Date,unit='ms')
    df['Actual_act'] = 'None'
    df['log_budget'] = budget
    df['Date'] = pd.to_datetime(df['Date'])
    stop_loss_percent = 0.005
    first_date = 0
    last_date = 0
    start_budget = budget
    final_budget = budget + 0
    start_coin = 0
    final_coin = 0
    current_coin = 0
    win_count = 0
    lose_count = 0
    round_buy = 0
    holding = False
    current = 'Hold'
    current_fee = 0
    cumulated_fee = 0
    skip = True
    for i in range(len(train_predict)+(look_back*2)+1,len(df)-2):
        if budget != 0:
            df.loc[i,['log_budget']] = budget
        else:
            df.loc[i,['log_budget']] = current_coin * df.iloc[i]['Open']
        if df.iloc[i][focus_column] == 'Buy' and holding == False and df.iloc[i]['Confident'] >= confident:
            round_buy += 1
            holding = True
            df.loc[i,['Actual_act']] = 'Buy'
            df.loc[i,['log_budget']] = budget
            current_fee = budget * fee
            current_coin = (budget - current_fee) /df.iloc[i]['Open']
            buy_at_price = df.iloc[i]['Open']
            if round_buy == 1:
                start_coin = current_coin
                first_date = df.iloc[i]['Date']
            print('I buy {} BTC with {} USD at price {} USD at {}'.format(current_coin,budget,buy_at_price\
                                                                         ,df.iloc[i]['Date']))
            print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            budget_temp = budget
            budget = 0
        elif df.iloc[i][focus_column] == 'Sell' and holding == True and df.iloc[i]['Confident'] >= confident\
                and (((current_coin * df.iloc[i]['Open']) - \
                     ((current_coin * df.iloc[i]['Open']) * fee)) >= buy_at_price * current_coin):
            print('do 1')
            
            holding = False
            df.loc[i,['Actual_act']] = 'Sell'
            current_fee = (current_coin * df.iloc[i]['Open']) * fee
            budget = current_coin * df.iloc[i]['Open'] - current_fee
            df.loc[i,['log_budget']] = budget
            sell_at_price = df.iloc[i]['Open']
            print('I sell {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin,\
                                        budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
            print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            final_budget += budget-budget_temp
            last_date = df.iloc[i]['Date']
            if budget < budget_temp:
                lose_count += 1
            else:
                win_count += 1
            current_coin = 0
            budget_temp = 0
        elif df.iloc[i][focus_column] == 'Sell' and holding == True and df.iloc[i]['Confident'] >= confident:
            if not skip:
                print('do 2')
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Open'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Open']
                print('I sell {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin,\
                                            budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
                skip = True
            else:
                print('skipped')
                skip = False
        elif df.iloc[i][focus_column] == 'Hold' and holding == True and enable_stop_loss:
            if df.iloc[i]['Open'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Open'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Open']
                print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
            elif df.iloc[i]['Close'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Close'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Close']
                print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
                
                
                                     
    final_coin = final_budget/df.iloc[len(df)-3]['Open']
    number_days = int(str(last_date-first_date)[:3])
    prof_percent = ((final_budget-start_budget)/start_budget)*100
    rel_prof_percent = ((final_coin-start_coin)/start_coin)*100
    if number_days > 30:
        days = number_days
        weeks = int(number_days/7)
        months = int(number_days/30)
    else:
        days = number_days
        weeks = int(number_days/7)
        months = 1
    print(final_coin,start_coin)
    print('Result : ')
    print('Start : {} Stop : {} = {} days'.format(first_date,last_date,number_days))
    print('Final Profit : {} USD = {} %'.format(final_budget-start_budget,((final_budget-start_budget)/start_budget)*100))
    print('Average Profit : /day = {} % /week = {} % /month = {} %'.format(prof_percent/days,\
         prof_percent/weeks,prof_percent/months))
    print('Exponential Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                10**(m.log(final_budget/start_budget,10)/days)*100-100,\
                                10**(m.log(final_budget/start_budget,10)/weeks)*100-100,
                                10**(m.log(final_budget/start_budget,10)/months)*100-100))
    print('Relative Final Profit : {} BTC = {} %'.format(final_coin-start_coin,((final_coin-start_coin)/start_coin)*100))
    print('Average Relative Profit : /day = {} % /week = {} % /month = {} %'.format(rel_prof_percent/days,\
         rel_prof_percent/weeks,rel_prof_percent/months))
    print('Exponential Relative Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                10**(m.log(final_coin/start_coin,10)/days)*100-100,\
                                10**(m.log(final_coin/start_coin,10)/weeks)*100-100,
                                10**(m.log(final_coin/start_coin,10)/months)*100-100))
    print('Win : {} Lose : {} Win rate : {} %'.format(win_count,lose_count,win_count/(win_count+lose_count)*100))
    print('Cumulated Fee : {}'.format(cumulated_fee))       
        
            
        
    

In [3]:
def testProfit2(df,budget,confident,focus_column,start,stop,enable_stop_loss=False,calculate_fee=True,\
                has_confident_column=True,return_profit=False,verbose=True):
    #buy, hold and sell only
    if calculate_fee:
        fee = 0.001
    else:
        fee = 0
    if not has_confident_column:
        print('Set Confident Column to 1')
        df['Confident'] = 1
    #df['Date'] = pd.to_datetime(df.Date,unit='ms')
    df['Actual_act'] = 'None'
    df['log_budget'] = budget
    df['Date'] = pd.to_datetime(df['Date'])
    df['Index_Buy'] = 0
    stop_loss_percent = 0.005
    first_date = 0
    last_date = 0
    start_budget = budget
    final_budget = budget + 0
    start_coin = 0
    final_coin = 0
    current_coin = 0
    win_count = 0
    lose_count = 0
    round_buy = 0
    holding = False
    current = 'Hold'
    current_fee = 0
    cumulated_fee = 0
    index_buy = 0
    skip = True
    for i in range(start,stop):
        if budget != 0:
            df.loc[i,['log_budget']] = budget
        else:
            df.loc[i,['log_budget']] = current_coin * df.iloc[i]['Open']
        if df.iloc[i][focus_column] == 'Buy' and holding == False and df.iloc[i]['Confident'] >= confident:
            round_buy += 1
            holding = True
            df.loc[i,['Actual_act']] = 'Buy'
            df.loc[i,['log_budget']] = budget
            current_fee = budget * fee
            current_coin = (budget - current_fee) /df.iloc[i]['Open']
            buy_at_price = df.iloc[i]['Open']
            if round_buy == 1:
                start_coin = current_coin
                first_date = df.iloc[i]['Date']
            if verbose:
                print('I buy {} BTC with {} USD at price {} USD at {}'.format(current_coin,budget,buy_at_price\
                                                                         ,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            budget_temp = budget
            budget = 0
            index_buy = i
        elif df.iloc[i][focus_column] == 'Sell' and holding == True and df.iloc[i]['Confident'] >= confident:
            holding = False
            df.loc[i,['Actual_act']] = 'Sell'
            current_fee = (current_coin * df.iloc[i]['Open']) * fee
            budget = current_coin * df.iloc[i]['Open'] - current_fee
            df.loc[i,['log_budget']] = budget
            sell_at_price = df.iloc[i]['Open']
            if verbose:
                print('I sell {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin,\
                                        budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            final_budget += budget-budget_temp
            last_date = df.iloc[i]['Date']
            if budget < budget_temp:
                lose_count += 1
                df.loc[i,['Actual_act']] = 'Lose'
                df.loc[i,['Index_Buy']] = index_buy
            else:
                win_count += 1
                df.loc[i,['Actual_act']] = 'Win'
            current_coin = 0
            budget_temp = 0
        elif df.iloc[i][focus_column] == 'Hold' and holding == True and enable_stop_loss:
            if df.iloc[i]['Open'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Open'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Open']
                if verbose:
                    print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                    print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                    df.loc[i,['Actual_act']] = 'Lose'
                    df.loc[i,['Index_Buy']] = index_buy
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
            elif df.iloc[i]['Close'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Close'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Close']
                if verbose:
                    print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                    print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                    df.loc[i,['Actual_act']] = 'Lose'
                    df.loc[i,['Index_Buy']] = index_buy
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
                
                
                                     
    final_coin = final_budget/df.iloc[len(df)-3]['Open']
    number_days = int(str(last_date-first_date)[:3])
    prof_percent = ((final_budget-start_budget)/start_budget)*100
    rel_prof_percent = ((final_coin-start_coin)/start_coin)*100
    if number_days > 30:
        days = number_days
        weeks = int(number_days/7)
        months = int(number_days/30)
    else:
        days = number_days
        weeks = int(number_days/7)
        months = 1
    if verbose:
        print()
        print('Result : ')
        print('Start : {} Stop : {} = {} days'.format(first_date,last_date,number_days))
        print('Final Profit : {} USD = {} %'.format(final_budget-start_budget,((final_budget-start_budget)/start_budget)*100))
        print('Average Profit : /day = {} % /week = {} % /month = {} %'.format(prof_percent/days,\
             prof_percent/weeks,prof_percent/months))
        print('Exponential Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                    10**(m.log(final_budget/start_budget,10)/days)*100-100,\
                                    10**(m.log(final_budget/start_budget,10)/weeks)*100-100,
                                    10**(m.log(final_budget/start_budget,10)/months)*100-100))
        print('Relative Final Profit : {} BTC = {} %'.format(final_coin-start_coin,((final_coin-start_coin)/start_coin)*100))
        print('Average Relative Profit : /day = {} % /week = {} % /month = {} %'.format(rel_prof_percent/days,\
             rel_prof_percent/weeks,rel_prof_percent/months))
        print('Exponential Relative Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                    10**(m.log(final_coin/start_coin,10)/days)*100-100,\
                                    10**(m.log(final_coin/start_coin,10)/weeks)*100-100,
                                    10**(m.log(final_coin/start_coin,10)/months)*100-100))
        print('Win : {} Lose : {} Win rate : {} %'.format(win_count,lose_count,win_count/(win_count+lose_count)*100))
        print('Cumulated Fee : {}'.format(cumulated_fee))       
        
    if return_profit:
        return final_budget-start_budget
        

In [2]:
def testProfit3(df,budget,confident,focus_column,start,stop,enable_stop_loss=False,calculate_fee=True,\
                has_confident_column=True,return_profit=False,verbose=True):
    #buy, hold and sell only
    if calculate_fee:
        fee = 0.001
    else:
        fee = 0
    if not has_confident_column:
        print('Set Confident Column to 1')
        df['Confident'] = 1
    #df['Date'] = pd.to_datetime(df.Date,unit='ms')
    df['Actual_act'] = 'None'
    df['log_budget'] = budget
    df['Date'] = pd.to_datetime(df['Date'])
    df['Index_Buy'] = 0
    stop_loss_percent = 0.005
    first_date = 0
    last_date = 0
    start_budget = budget
    final_budget = budget + 0
    start_coin = 0
    final_coin = 0
    current_coin = 0
    win_count = 0
    lose_count = 0
    round_buy = 0
    holding = False
    current = 'Hold'
    current_fee = 0
    cumulated_fee = 0
    index_buy = 0
    skip = True
    #DeMark9
    count_buy = 0
    count_sell = 0
    demark_buy = False
    demark_sell = False
    
    for i in range(start,stop):
        if i >= 4:
            if df.iloc[i-4]['Close'] > df.iloc[i]['Close']:
                count_sell = 0
                count_buy += 1
                
                if count_buy == 9:
                    count_buy = 0
                    demark_buy = True
            else:
                count_buy = 0
                count_sell += 1
                
                if count_sell == 9:
                    count_sell = 0
                    demark_sell = True
        if budget != 0:
            df.loc[i,['log_budget']] = budget
        else:
            df.loc[i,['log_budget']] = current_coin * df.iloc[i]['Open']
        if df.iloc[i][focus_column] == 'Buy' and holding == False and df.iloc[i]['Confident'] >= confident\
            and demark_buy:
            round_buy += 1
            holding = True
            df.loc[i,['Actual_act']] = 'Buy'
            df.loc[i,['log_budget']] = budget
            current_fee = budget * fee
            current_coin = (budget - current_fee) /df.iloc[i]['Open']
            buy_at_price = df.iloc[i]['Open']
            if round_buy == 1:
                start_coin = current_coin
                first_date = df.iloc[i]['Date']
            if verbose:
                print('I buy {} BTC with {} USD at price {} USD at {}'.format(current_coin,budget,buy_at_price\
                                                                         ,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            budget_temp = budget
            budget = 0
            index_buy = i
            demark_buy = False
        elif df.iloc[i][focus_column] == 'Sell' and holding == True and df.iloc[i]['Confident'] >= confident\
            and demark_sell:
            holding = False
            df.loc[i,['Actual_act']] = 'Sell'
            current_fee = (current_coin * df.iloc[i]['Open']) * fee
            budget = current_coin * df.iloc[i]['Open'] - current_fee
            df.loc[i,['log_budget']] = budget
            sell_at_price = df.iloc[i]['Open']
            if verbose:
                print('I sell {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin,\
                                        budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                print('Fee : {} USD'.format(current_fee))
            cumulated_fee += current_fee
            final_budget += budget-budget_temp
            last_date = df.iloc[i]['Date']
            if budget < budget_temp:
                lose_count += 1
                df.loc[i,['Actual_act']] = 'Lose'
                df.loc[i,['Index_Buy']] = index_buy
            else:
                win_count += 1
                df.loc[i,['Actual_act']] = 'Win'
            current_coin = 0
            budget_temp = 0
            demark_sell = False
        elif df.iloc[i][focus_column] == 'Hold' and holding == True and enable_stop_loss:
            if df.iloc[i]['Open'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Open'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Open']
                if verbose:
                    print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                    print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                    df.loc[i,['Actual_act']] = 'Lose'
                    df.loc[i,['Index_Buy']] = index_buy
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
            elif df.iloc[i]['Close'] < (buy_at_price-(stop_loss_percent * buy_at_price)):
                holding = False
                df.loc[i,['Actual_act']] = 'Sell'
                current_fee = (current_coin * df.iloc[i]['Open']) * fee
                budget = current_coin * df.iloc[i]['Close'] - current_fee
                df.loc[i,['log_budget']] = budget
                sell_at_price = df.iloc[i]['Close']
                if verbose:
                    print('I stop loss {} BTC and get profit {} USD at price {} USD at {}'.format(current_coin\
                                                    ,budget-budget_temp,sell_at_price,df.iloc[i]['Date']))
                    print('Fee : {} USD'.format(current_fee))
                cumulated_fee += current_fee
                final_budget += budget-budget_temp
                last_date = df.iloc[i]['Date']
                if budget < budget_temp:
                    lose_count += 1
                    df.loc[i,['Actual_act']] = 'Lose'
                    df.loc[i,['Index_Buy']] = index_buy
                else:
                    win_count += 1
                current_coin = 0
                budget_temp = 0
                
                
                                     
    final_coin = final_budget/df.iloc[len(df)-3]['Open']
    number_days = int(str(last_date-first_date)[:3])
    prof_percent = ((final_budget-start_budget)/start_budget)*100
    rel_prof_percent = ((final_coin-start_coin)/start_coin)*100
    if number_days > 30:
        days = number_days
        weeks = int(number_days/7)
        months = int(number_days/30)
    else:
        days = number_days
        weeks = int(number_days/7)
        months = 1
    if verbose:
        print()
        print('Result : ')
        print('Start : {} Stop : {} = {} days'.format(first_date,last_date,number_days))
        print('Final Profit : {} USD = {} %'.format(final_budget-start_budget,((final_budget-start_budget)/start_budget)*100))
        print('Average Profit : /day = {} % /week = {} % /month = {} %'.format(prof_percent/days,\
             prof_percent/weeks,prof_percent/months))
        print('Exponential Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                    10**(m.log(final_budget/start_budget,10)/days)*100-100,\
                                    10**(m.log(final_budget/start_budget,10)/weeks)*100-100,
                                    10**(m.log(final_budget/start_budget,10)/months)*100-100))
        print('Relative Final Profit : {} BTC = {} %'.format(final_coin-start_coin,((final_coin-start_coin)/start_coin)*100))
        print('Average Relative Profit : /day = {} % /week = {} % /month = {} %'.format(rel_prof_percent/days,\
             rel_prof_percent/weeks,rel_prof_percent/months))
        print('Exponential Relative Profit : /d = {} % /week = {} % /month = {} %'.format(\
                                    10**(m.log(final_coin/start_coin,10)/days)*100-100,\
                                    10**(m.log(final_coin/start_coin,10)/weeks)*100-100,
                                    10**(m.log(final_coin/start_coin,10)/months)*100-100))
        print('Win : {} Lose : {} Win rate : {} %'.format(win_count,lose_count,win_count/(win_count+lose_count)*100))
        print('Cumulated Fee : {}'.format(cumulated_fee))       
        
    if return_profit:
        return final_budget-start_budget
        

In [ ]:
def significantMark(df,mask_threshold):
    mask_directory = 'New_Dataset/Completed/15_55_BTCUSD_1D_2019_2021_t'+str(mask_threshold)+'_spread_0.csv'
    df_mask = pd.read_csv(mask_directory)
    
    df['Date'] = df['Date'].apply(lambda x: dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df_mask['Date'] = df_mask['Date'].apply(lambda x: dtime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    if df_mask.Date[0] > df.Date[0]:
        df = df[df.Date >= df_mask.Date[0]]
        df.reset_index(drop=True,inplace=True)
    else:
        df_mask = df_mask[df_mask.Date >= df.Date[0]]
        df_mask.reset_index(drop=True,inplace=True)
        
    #masking
    df['Significant_Change'] = 0
    current_index = 0
    found = False
    for i,row in df_mask[df_mask.Significant_Change == 1].iterrows():
        day = row.Date.day
        month = row.Date.month
        year = row.Date.year
        print(current_index)
        for k,line in df[current_index:].iterrows():
            if line.Date.day == day and line.Date.month == month and line.Date.year == year:
                print(f'At index: {k}')
                current_index = k
                found = True
                df.loc[k,['Significant_Change']] = 1
            if k>current_index and found == True:
                print(f'Stop at index {k}')
                found = False
                break
    return df